In [15]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [16]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [17]:
# default_exp agents.dqn.core

In [18]:
# export
# Python native modules
import os
from collections import deque
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block_simple import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *

# DQN Core
> Contains the minimum DQN API.

In [19]:
# export
class DQN(Module):
    def __init__(self,state_sz:int,action_sz:int,hidden=512):
        self.layers=Sequential(
            Linear(state_sz,hidden),
            ReLU(),
            Linear(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x.float())

In [20]:
dqn=DQN(4,2)
dqn(torch.randn((10,4)))

tensor([[-0.1770, -0.1525],
        [ 0.0488, -0.1218],
        [ 0.1171, -0.1276],
        [-0.0862, -0.1140],
        [-0.2271, -0.2124],
        [-0.1201,  0.1057],
        [-0.0687, -0.1929],
        [-0.1460, -0.0510],
        [-0.3315, -0.3717],
        [-0.1599, -0.0437]], grad_fn=<AddmmBackward>)

In [21]:
# export
class ArgMaxFeed(AgentCallback):
    def before_action(self): 
        raw_action=self.agent.model(self.experience['state'].to(default_device()))
        self.agent.raw_action_shape=raw_action.shape
        self.agent.action=torch.argmax(raw_action,dim=1).reshape(-1,1)
        
class DiscreteEpsilonRandomSelect(AgentCallback):
    
    def __init__(self,idx=0,min_epsilon=0.2,max_epsilon=1,max_steps=5000):
        store_attr()
        self.epsilon=max_epsilon
    
    def before_noise(self): 
        self.mask=torch.randn(size=(self.agent.action.shape[0],))<self.epsilon
        self.experience['randomly_selected']=self.mask.reshape(-1,1)
        self.experience['epsilon']=torch.full(self.agent.action.shape,self.epsilon)
        self.experience['orignal_actions']=self.agent.action.detach().clone()
        self.agent.action[self.mask]=self.agent.action[self.mask].random_(0,self.agent.raw_action_shape[1])
        self.agent.action=self.agent.action.detach().cpu().numpy()
    
        if self.agent.model.training: 
            self.idx+=1
            self.epsilon=max(self.min_epsilon,self.max_epsilon-self.idx/self.max_steps)

In [22]:
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(max_epsilon=0.5)]).to(default_device())
agent.do_action(state=torch.randn((10,4)).to(default_device()))


(array([[1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0]]),
 {'state': tensor([[-0.3860, -0.3133,  1.7219, -0.8883],
          [ 0.9461,  0.7227, -0.9253, -0.5611],
          [ 0.2669, -0.1310,  0.4252,  2.3886],
          [-0.5233,  1.1551, -1.1339, -1.4867],
          [ 0.6150, -0.0886,  0.0781,  0.8859],
          [ 1.8412, -0.7758, -0.7202, -0.9623],
          [-1.9321, -0.3855,  0.1164, -1.9733],
          [-0.1910,  1.3213, -0.6909,  0.8464],
          [-1.0414,  0.1258,  0.5049, -0.1680],
          [-0.4623, -1.2743, -0.2980, -0.2412]], device='cuda:0'),
  'randomly_selected': tensor([[ True],
          [False],
          [ True],
          [False],
          [ True],
          [ True],
          [ True],
          [False],
          [ True],
          [ True]]),
  'epsilon': tensor([[0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.500

In [23]:
agent.show_loop()

Start Action
   - before_preprocess: []
   - after_preprocess: []
   - before_action  : [ArgMaxFeed]
   - after_action   : []
   - before_noise   : [DiscreteEpsilonRandomSelect]
   - after_noise    : []
End Action


In [24]:
# export        
class Epsilon(Metric):
    order=30
    epsilon=0

    @property
    def value(self): return self.epsilon
    def reset(self): self.epsilon=0
    def accumulate(self,learn):
        for cb in learn.model.cbs:
            if type(cb)==DiscreteEpsilonRandomSelect:
                self.epsilon=cb.epsilon

In [56]:
# export
class ExperienceReplay(Callback):
    def __init__(self,*args,bs=16,max_sz=200,warmup_sz=100,**kwargs):
        store_attr()
        self.memory=None
        self.pointer=0
        self.cache=None
    
    def after_pred(self):
        xb=BD(self.learn.xb[0]).mapv(to_detach)
        self.cache=xb
        if self.memory is None:            self.memory=xb
        elif self.memory.bs()<self.max_sz: self.memory+=xb
        else:
            self.memory=self.memory[:self.pointer]+xb+self.memory[self.pointer+xb.bs()+1:]
            self.pointer+=xb.bs()
            if self.pointer>self.max_sz: self.pointer=0
        with torch.no_grad():
            idxs=np.random.randint(0,self.memory.bs(),self.bs).tolist()
            self.learn.xb=(self.memory[idxs].mapv(to_device),)
        
        if self.memory.bs()<self.warmup_sz: raise CancelBatchException
        
    def after_batch(self): self.learn.xb=self.cache

In [57]:
# export
class DQNTrainer(Callback):
    "Performs traditional training on `next_q`. Requires a callback such as `RegularNextQ`"
    def __init__(self,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
    
    def after_pred(self): 
        self.learn.yb=self.xb
        self.learn.xb=self.xb[0]
        self._xb=({k:v.clone() for k,v in self.xb.items()},)
        self.learn.done_mask=self.xb['done'].reshape(-1,)
        
        # Get targets
        self.learn.next_q=self.learn.model.model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
        self.learn.next_q[self.done_mask]=0 #xb[done_mask]['reward']
        self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
        self.learn.yb=(self.learn.targets.reshape(-1),)
        
        # Get prediction
        self.learn.action_v=self.learn.model.model(self.xb['state'])
        self.learn.pred=self.learn.action_v.gather(1,self.xb['action']).reshape(-1)
        
    def before_backward(self): self.learn.xb=self._xb

In [58]:
# slow
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=1,seed=0,
                           steps_delta=1,),FirstLast])

dls=SourceDataBlock().dataloaders([source],n=1000,bs=1,num_workers=0)

exp_replay=ExperienceReplay(bs=32,max_sz=100000,warmup_sz=32)

learn=Learner(dls,agent,loss_func=MSELoss(),
              cbs=[exp_replay,DQNTrainer],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [59]:
learn.fit(5,lr=0.0001,wd=0)

> Note: This should work without a exp replay although will perform poorly.

In [60]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.block_simple.ipynb.
Converted 05c_data.gym.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/10a_agents.dqn.core.ipynb
